Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [9]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [10]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [11]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (10000, 28, 28, 1), (10000, 10))
('Test set', (10000, 28, 28, 1), (10000, 10))


In [12]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.806191
Minibatch accuracy: 6.2%
Validation accuracy: 12.5%
Minibatch loss at step 50: 2.201069
Minibatch accuracy: 37.5%
Validation accuracy: 44.1%
Minibatch loss at step 100: 1.361092
Minibatch accuracy: 62.5%
Validation accuracy: 67.6%
Minibatch loss at step 150: 0.656549
Minibatch accuracy: 87.5%
Validation accuracy: 75.2%
Minibatch loss at step 200: 0.933998
Minibatch accuracy: 75.0%
Validation accuracy: 74.7%
Minibatch loss at step 250: 1.153798
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%
Minibatch loss at step 300: 0.890163
Minibatch accuracy: 75.0%
Validation accuracy: 77.3%
Minibatch loss at step 350: 1.045606
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.429543
Minibatch accuracy: 87.5%
Validation accuracy: 79.2%
Minibatch loss at step 450: 0.739318
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 500: 0.716841
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Mi

---
MODIFIED VERSION WITH TENSORBOARD SCOPES (DAY)
------------------------------------------

This does the same as the model above, but I have added scoping and summaries so you can look
at the model with tensorboard.

---

In [16]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
num_steps = 1001

graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('train') as trainscope:
        tf_train_dataset = tf.placeholder(
            tf.float32, shape=(batch_size, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    with tf.name_scope('validate') as validatescope:
        tf_valid_dataset = tf.constant(valid_dataset)
    with tf.name_scope('test') as testscope:
        tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    with tf.name_scope('layer1'):
        layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
        layer1_biases = tf.Variable(tf.zeros([depth]))
    with tf.name_scope('layer2'):
        layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    with tf.name_scope('layer3'):
        layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    with tf.name_scope('layer4'):
        layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        with tf.name_scope('convolution1'):
            conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            # hidden will be 28/2=14 width, 14 height, 16 depth
        with tf.name_scope('convolution2'):
            conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer2_biases)
            # this hidden will be 14/2=7 width, 7 height, 16 depth
        with tf.name_scope('fully_connected1'):
            shape = hidden.get_shape().as_list()
            print('Shape of hidden is %s' % shape)
            reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        with tf.name_scope('fully_connected2'):
            result = tf.matmul(hidden, layer4_weights) + layer4_biases
        return result
  
    # Training computation and optimizer
    with tf.name_scope(trainscope):
        logits = model(tf_train_dataset)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        tf.summary.scalar('loss', loss)
    
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    with tf.name_scope(trainscope):
        train_prediction = tf.nn.softmax(logits)
    with tf.name_scope(validatescope):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    with tf.name_scope(testscope):
        test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

with tf.Session(graph=graph) as session:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('/tmp/mnist_logs', graph=session.graph)
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, merged_output = session.run(
            [optimizer, loss, train_prediction, merged], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
            writer.add_summary(merged_output)
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Shape of hidden is [16, 7, 7, 16]
Shape of hidden is [10000, 7, 7, 16]
Shape of hidden is [10000, 7, 7, 16]
Initialized
Minibatch loss at step 0: 4.380046
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 50: 2.229903
Minibatch accuracy: 31.2%
Validation accuracy: 49.8%
Minibatch loss at step 100: 1.513548
Minibatch accuracy: 62.5%
Validation accuracy: 62.8%
Minibatch loss at step 150: 0.643905
Minibatch accuracy: 87.5%
Validation accuracy: 74.6%
Minibatch loss at step 200: 0.869323
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%
Minibatch loss at step 250: 1.065053
Minibatch accuracy: 75.0%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.720386
Minibatch accuracy: 81.2%
Validation accuracy: 76.5%
Minibatch loss at step 350: 1.071329
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 400: 0.414504
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 450: 0.538753
Minibatch accuracy: 87.5%
Validation 

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Answer 1
===
I interpreted this literally as replacing the convolutions with max_pools, even though the more typical setup would be to do a convolution followed by a pooling.  In the lesson, the teacher says that convolution with a stride of 2 is a fairly drastic way to reduce the dimensions of the input since it loses data and that pooling can be a more successful way to reduce the dimensions.  I suspect that I'll want to convolute with a stride of 1 followed by pooling with a stride of 2.

---

In [18]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
num_steps = 1001

graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('train') as trainscope:
        tf_train_dataset = tf.placeholder(
            tf.float32, shape=(batch_size, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    with tf.name_scope('validate') as validatescope:
        tf_valid_dataset = tf.constant(valid_dataset)
    with tf.name_scope('test') as testscope:
        tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These first two layers don't have weights and biases when we use just pooling
    #with tf.name_scope('layer1'):
    #    layer1_weights = tf.Variable(tf.truncated_normal(
    #        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    #    layer1_biases = tf.Variable(tf.zeros([depth]))
    #with tf.name_scope('layer2'):
    #    layer2_weights = tf.Variable(tf.truncated_normal(
    #        [patch_size, patch_size, depth, depth], stddev=0.1))
    #    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    with tf.name_scope('layer3'):
        # Since we are doing max_pool instead of a convnet step, the resulting
        # depth of the shape will be the same as the original input (num_channels)
        # rather than the arbitrary depth we had specified for the conv (depth)
        #layer3_weights = tf.Variable(tf.truncated_normal(
        #    [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
        layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * num_channels, num_hidden], stddev=0.1))
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    with tf.name_scope('layer4'):
        layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        # We would normally alternate a conv with a pool, but this assignment asks to replace
        # the conv with a pool.
        with tf.name_scope('pool1'):
            pool = tf.nn.max_pool(data, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            # pool will be 14x14x1 I think (14 width, 14 height, 1 depth [or channels])
        with tf.name_scope('pool2'):
            pool2 = tf.nn.max_pool(pool, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        with tf.name_scope('fully_connected1'):
            shape = pool2.get_shape().as_list()
            reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        with tf.name_scope('fully_connected2'):
            result = tf.matmul(hidden, layer4_weights) + layer4_biases
        return result
  
    # Training computation and optimizer
    with tf.name_scope(trainscope):
        logits = model(tf_train_dataset)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        tf.summary.scalar('loss', loss)
    
        optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    with tf.name_scope(trainscope):
        train_prediction = tf.nn.softmax(logits)
    with tf.name_scope(validatescope):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    with tf.name_scope(testscope):
        test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

with tf.Session(graph=graph) as session:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('/tmp/mnist_logs', graph=session.graph)
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, merged_output = session.run(
            [optimizer, loss, train_prediction, merged], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
            writer.add_summary(merged_output)
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.559013
Minibatch accuracy: 18.8%
Validation accuracy: 11.7%
Minibatch loss at step 50: 2.235956
Minibatch accuracy: 37.5%
Validation accuracy: 49.6%
Minibatch loss at step 100: 1.828106
Minibatch accuracy: 50.0%
Validation accuracy: 55.9%
Minibatch loss at step 150: 1.195057
Minibatch accuracy: 68.8%
Validation accuracy: 64.9%
Minibatch loss at step 200: 1.555825
Minibatch accuracy: 25.0%
Validation accuracy: 67.1%
Minibatch loss at step 250: 1.114750
Minibatch accuracy: 81.2%
Validation accuracy: 71.4%
Minibatch loss at step 300: 1.097507
Minibatch accuracy: 75.0%
Validation accuracy: 71.7%
Minibatch loss at step 350: 1.076211
Minibatch accuracy: 68.8%
Validation accuracy: 72.4%
Minibatch loss at step 400: 0.688659
Minibatch accuracy: 75.0%
Validation accuracy: 74.2%
Minibatch loss at step 450: 0.815226
Minibatch accuracy: 81.2%
Validation accuracy: 73.7%
Minibatch loss at step 500: 1.161875
Minibatch accuracy: 75.0%
Validation accuracy: 73.7%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

---
Answer 2
====
I implemented the following architecture and got 92.3%

1a. convolution patch=5, stride=1, padding=same, depth=16

1b. pool kernel(shape)=2x2, stride=2

2a. convolution patch=5, stride=1, padding=same, depth=32

2b. pool kernel=2, stride=2

3. fully connected layer (like lesson 3) inputdepth=32, outputdepth=64

4. (logits) fully connected layer inputdepth=64, outputdepth=num_labels

I did 2001 steps at a batch size of 16.  I did not implement training decay (and started with 0.05).  I initialized the weights with stdev of 1e-1 (this was a sensitive hyperparameter, if I made it too small, the validation accuracy never got below 10% (random))

---

In [20]:
batch_size = 16
patch_size = 5
depth1 = 16
depth2 = depth1*2
num_hidden = 64
num_steps = 2001
learning_rate_no_decay = 0.05
starter_learning_rate = 0.5
decay_steps = 100
decay_rate = 0.96
weight_initialization_stddev = 1e-1


graph = tf.Graph()

with graph.as_default():

    # Input data.
    with tf.name_scope('train') as trainscope:
        tf_train_dataset = tf.placeholder(
            tf.float32, shape=(batch_size, image_size, image_size, num_channels))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    with tf.name_scope('validate') as validatescope:
        tf_valid_dataset = tf.constant(valid_dataset)
    with tf.name_scope('test') as testscope:
        tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    with tf.name_scope('layer1'):
        layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth1], stddev=weight_initialization_stddev))
        layer1_biases = tf.Variable(tf.zeros([depth1]))
    with tf.name_scope('layer2'):
        # this is still OK for the dimensions for layer 2 after adding the pool.
        layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth1, depth2], stddev=weight_initialization_stddev))
        layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
    with tf.name_scope('layer3'):
        layer3_weights = tf.Variable(tf.truncated_normal(
            [image_size // 4 * image_size // 4 * depth2, num_hidden], stddev=weight_initialization_stddev))
        layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    with tf.name_scope('layer4'):
        layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=weight_initialization_stddev))
        layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        with tf.name_scope('conv_pool1'):
            # DAY use a stride of 1 on the convolutions and depend instead on 
            #   a pooling afterwards at stride of 2 to reduce the dimensions
            # DAY conv2d takes an input of shape [batch_size, height, width, channels]
            conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
            hidden = tf.nn.relu(conv + layer1_biases)
            # hidden will be 16 batch, 28/1=28 width, 28 height, 16 depth
            # Now add a pooling to reduce the dimensions.
            pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
            # pool will be 16 batch, 28/2=14 width, 14 height, 16 depth
        with tf.name_scope('conv_pool2'):
            conv2 = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
            hidden2 = tf.nn.relu(conv2 + layer2_biases)
            # this hidden will be 16 batch, 14/1=14 width, 14 height, 16*2=32 depth
            # Now add a pooling to reduce the dimensions.
            pool2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
            # pool will be 16 batch, 14/2=7 width, 7 height, 32 depth
        with tf.name_scope('fully_connected1'):
            shape = pool2.get_shape().as_list()
            print('Shape of pool is %s' % shape)
            reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
            hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        with tf.name_scope('fully_connected2'):
            result = tf.matmul(hidden3, layer4_weights) + layer4_biases
        return result
  
    # Training computation and optimizer
    with tf.name_scope(trainscope):
        logits = model(tf_train_dataset)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        # DAY: TODO try adding a term to the loss called regularization
        tf.summary.scalar('loss', loss)
    
        global_step = tf.Variable(0)  # count the number of steps taken.
        #learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 
        #                                                decay_steps, decay_rate, staircase=True)
        #tf.scalar_summary('learning_rate', learning_rate)

        #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate_no_decay).minimize(loss, 
                                                                                   global_step=global_step)

    # Predictions for the training, validation, and test data.
    with tf.name_scope(trainscope):
        train_prediction = tf.nn.softmax(logits)
    with tf.name_scope(validatescope):
        valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    with tf.name_scope(testscope):
        test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
    with tf.name_scope('Accuracy_Calculation'):
        train_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_prediction,1), 
                                                         tf.argmax(tf_train_labels,1)), tf.float32))
        #valid_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(valid_prediction,1), 
        #                                                tf.argmax(valid_labels,1)), tf.float32))
        #test_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(test_prediction,1), 
        #                                                tf.argmax(test_labels,1)), tf.float32))
        train_acc_sum = tf.summary.scalar('training_set_accuracy', train_accuracy)
        #valid_acc_sum = tf.scalar_summary('validation set accuracy', valid_accuracy)
        #test_acc_sum = tf.scalar_summary('test set accuracy', test_accuracy)


with tf.Session(graph=graph) as session:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('/tmp/mnist_logs', graph=session.graph)
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        global_step = step
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, merged_output = session.run(
            [optimizer, loss, train_prediction, merged], feed_dict=feed_dict)
        if(step %9 == 0):
            #print('.')
            pass
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
            writer.add_summary(merged_output)
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Shape of pool is [16, 7, 7, 32]
Shape of pool is [10000, 7, 7, 32]
Shape of pool is [10000, 7, 7, 32]
Initialized
Minibatch loss at step 0: 4.847366
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.297771
Minibatch accuracy: 18.8%
Validation accuracy: 25.1%
Minibatch loss at step 100: 2.186875
Minibatch accuracy: 56.2%
Validation accuracy: 51.6%
Minibatch loss at step 150: 0.754239
Minibatch accuracy: 81.2%
Validation accuracy: 75.0%
Minibatch loss at step 200: 0.917852
Minibatch accuracy: 75.0%
Validation accuracy: 76.3%
Minibatch loss at step 250: 0.937996
Minibatch accuracy: 81.2%
Validation accuracy: 77.4%
Minibatch loss at step 300: 0.886154
Minibatch accuracy: 68.8%
Validation accuracy: 77.4%
Minibatch loss at step 350: 0.696308
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 400: 0.404180
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Minibatch loss at step 450: 0.463291
Minibatch accuracy: 87.5%
Validation accura